# Tutorial 2: Cycled experiment


**Goal**: To run a cycled data assimilation experiment.
[`cycled_exp.py`](https://github.com/lkugler/DART-WRF/blob/master/generate_free.py) contains an example which will be explained here:

#### Configure your experiment
See tutorial 1.

#### Prepare initial conditions from previous forecasts
Before starting, let's set up the directory structure with
```python
begin = dt.datetime(2008, 7, 30, 6)
prepare_WRFrundir(begin)
```

#### Run a forecast
Let's say you
- want to run a forecast starting at 6 UTC until 12 UTC
- do not want WRF restart files

then the required code is
```python
begin = dt.datetime(2008, 7, 30, 6)
end = dt.datetime(2008, 7, 30, 12)

id = run_ENS(begin=begin,  # start integration from here
             end=end,      # integrate until here
             output_restart_interval=9999,  # do not write WRF restart files
             depends_on=id)
```
Note that `begin` and `end` are `dt.datetime` objects.

#### Assimilate
To assimilate observations at dt.datetime `time` use this command:

`id = assimilate(time, prior_init_time, prior_valid_time, prior_path_exp, depends_on=id)`

#### Update initial conditions from Data Assimilation
In order to continue after assimilation you need the posterior = prior (1) + increments (2)

1. Set prior with this function:

`id = prepare_IC_from_prior(prior_path_exp, prior_init_time, prior_valid_time, depends_on=id)`

where path is `str`, times are `dt.datetime`.

2. To update the model state with assimilation increments, you need to update the WRF restart files by running

`id = update_IC_from_DA(time, depends_on=id)`

After this, the wrfrst files are updated with assimilation increments (filter_restart) and copied to the WRF's run directories so you can continue to run the ENS after assimilation using function `run_ENS()`.



#### Cycled data assimilation code

Then the loop looks like



#### Job scheduling status
The script submits jobs into the SLURM queue with dependencies so that SLURM starts the jobs itself as soon as resources are available. Most jobs need only a few cores, but model integration is done across many nodes:
```bash
$ squeue -u `whoami` --sort=i
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1710274  mem_0384 prepwrfr  lkugler PD       0:00      1 (Priority)
           1710275  mem_0384 IC-prior  lkugler PD       0:00      1 (Dependency)
           1710276  mem_0384 Assim-42  lkugler PD       0:00      1 (Dependency)
           1710277  mem_0384 IC-prior  lkugler PD       0:00      1 (Dependency)
           1710278  mem_0384 IC-updat  lkugler PD       0:00      1 (Dependency)
           1710279  mem_0384 preWRF2-  lkugler PD       0:00      1 (Dependency)
    1710280_[1-10]  mem_0384 runWRF2-  lkugler PD       0:00      1 (Dependency)
           1710281  mem_0384 pRTTOV-6  lkugler PD       0:00      1 (Dependency)
           1710282  mem_0384 Assim-3a  lkugler PD       0:00      1 (Dependency)
           1710283  mem_0384 IC-prior  lkugler PD       0:00      1 (Dependency)
           1710284  mem_0384 IC-updat  lkugler PD       0:00      1 (Dependency)
           1710285  mem_0384 preWRF2-  lkugler PD       0:00      1 (Dependency)
    1710286_[1-10]  mem_0384 runWRF2-  lkugler PD       0:00      1 (Dependency)
           1710287  mem_0384 pRTTOV-7  lkugler PD       0:00      1 (Dependency)
```

